# 🤝 휴먼 인더루프(Human-in-the-Loop) 패턴

> **💡 핵심 개념**  
> 에이전트가 중요한 작업을 실행하기 전에 사용자의 승인을 받는 패턴입니다.

### 📋 개요

에이전트가 함수 호출 승인과 같이 사용자 입력을 필요로 하는 경우, 이를 **'휴먼 인더루프(Human-in-the-Loop)'** 패턴이라고 합니다. 

#### 🔄 작동 방식
- 사용자 입력이 필요한 에이전트 실행은 최종 답변 대신 **필요한 입력이 무엇인지 알려주는 응답**으로 완료됩니다
- 에이전트 호출자는 사용자로부터 필요한 입력을 받아 **새로운 에이전트 실행의 일부로 다시 전달**해야 합니다

## 🛠️ 승인이 필요한 Function 도구 생성

### ✨ 승인 모드 설정

함수를 사용할 때, `@tool` 데코레이터의 **`approval_mode`** 매개변수를 활용하여 실행 전 승인 여부를 지정할 수 있습니다.

```python
approval_mode="always_require"  # 항상 승인 필요
```

### 📝 예제: 날씨 조회 도구

아래는 특정 위치의 날씨 정보를 가져오는 간단한 함수 도구 예시입니다.

In [ ]:
from typing import Annotated
from agent_framework import tool

@tool
def get_weather(location: Annotated[str, "장소 정보, 도시와 주, 예: 서울, 대한민국 📍"]) -> str:
    """주어진 위치의 현재 날씨를 가져옵니다. ☁️"""
    return f"{location}은 현재 구름이 많으며, 온도는 15°C 입니다."

In [ ]:
from typing import Annotated
from agent_framework import tool

@tool(approval_mode="always_require")
def get_weather_detail(location: Annotated[str, "장소 정보, 도시와 주, 예: 서울, 대한민국 📍"]) -> str:
    """주어진 위치의 상세한 날씨 정보를 가져옵니다. ⛅✨"""
    return f"{location}은 현재 구름이 많으며, 온도는 15°C, 습도는 88% 입니다."

## 🤖 에이전트에 승인 도구 통합

`ChatAgent` 생성자의 **`tools`** 매개변수에 도구 목록을 전달하여 승인이 필요한 함수 도구를 에이전트에 제공할 수 있습니다.

In [ ]:
from agent_framework import ChatAgent
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

agent = ChatAgent(
    chat_client=AzureOpenAIChatClient(credential=AzureCliCredential()),
    name="WeatherAgent",
    instructions=(
        "당신은 도움이 되는 날씨 어시스턴트입니다. ☀️💁"
    ),
    tools=[get_weather, get_weather_detail],
)
    
# 에이전트 실행
response = await agent.run("뉴욕의 날씨는 어떤가요?")
print(response.text)

## ✅ 승인 요청 처리하기

승인이 필요한 함수 도구를 호출하여 사용자 입력 요청이 발생하는지 확인해봅니다.

In [ ]:
result = await agent.run("서울의 상세한 날씨는 어떤가요?")

if result.user_input_requests:
    for user_input_needed in result.user_input_requests:
        print(f"Function: {user_input_needed.function_call.name}")
        print(f"Arguments: {user_input_needed.function_call.arguments}")

## 🔐 승인 프로세스 상세 설명

### 📌 승인 요청 감지

승인이 필요한 함수가 있는 경우, 에이전트는 함수를 **직접 실행하지 않고** 승인 요청을 보냅니다.

```python
if result.user_input_requests:  # 승인 요청 확인
```

### 📤 승인 정보 확인

함수 승인 요청이 있을 때, 다음 정보를 확인할 수 있습니다:
- **함수 이름**: `user_input_needed.function_call.name`
- **인수**: `user_input_needed.function_call.arguments`

### 👤 사용자 응답 생성

사용자가 승인 여부를 결정하면, `create_response` 메서드를 사용하여 응답을 생성합니다:
- ✅ **승인**: `True` 전달
- ❌ **거부**: `False` 전달

### 🔄 에이전트에 응답 전달

생성된 응답을 `ChatMessage`로 에이전트에 전달하여 결과를 받습니다.

In [ ]:
## 예시 코드입니다.
## 실제 코드는 다음 영역에서 계속됩니다.
from agent_framework import ChatMessage, Role

# Get user approval (in a real application, this would be interactive)
user_approval = True  # or False to reject

# Create the approval response
approval_message = ChatMessage(
    role=Role.USER, 
    contents=[user_input_needed.to_function_approval_response(user_approval)]
)

# Continue the conversation with the approval
final_result = await agent.run([
    "암스테르담의 상세한 날씨는 어떤가요?",
    ChatMessage(role=Role.ASSISTANT, contents=[user_input_needed]),
    approval_message
])
print(final_result.text)

## 🔁 반복적 승인 처리

### 💼 다중 함수 호출 처리

승인이 필요한 **여러 함수 호출**을 처리할 때는 모든 함수가 승인되거나 거부될 때까지 **반복문**을 사용하여 승인을 처리해야 합니다.

In [ ]:
from agent_framework import ChatMessage, Role

async def handle_approvals(query: str, agent) -> str:
    """반복문에서 함수 호출 승인을 처리합니다. ✅🔄"""
    current_input = query

    while True:
        result = await agent.run(current_input)

        if not result.user_input_requests:
            # 더이상 승인이 필요하지 않으므로 최종 결과를 반환합니다.
            return result.text

        # 모든 컨텍스트를 포함한 새 입력을 만듭니다.
        new_inputs = [query]

        for user_input_needed in result.user_input_requests:
            print(f"Approval needed for: {user_input_needed.function_call.name}")
            print(f"Arguments: {user_input_needed.function_call.arguments}")

            # 승인 요청을 포함하는 어시스턴트 메시지 추가
            new_inputs.append(ChatMessage(role=Role.ASSISTANT, contents=[user_input_needed]))

            # 사용자 승인 가져오기 (실제 애플리케이션에서는 상호작용 방식이어야 함)
            # 여기서는 모두 '승인'하는 것으로 가정합니다.
            user_approval = True  # 실제 사용자 입력으로 교체하세요

            # 사용자의 승인 응답을 추가합니다
            new_inputs.append(
                ChatMessage(role=Role.USER, contents=[user_input_needed.to_function_approval_response(user_approval)])
            )

        # 모든 컨텍스트를 포함해서 계속 진행합니다.
        current_input = new_inputs

# Usage
result_text = await handle_approvals("시애틀과 포틀랜드의 상세한 날씨 정보를 알려줘", agent)
print(result_text)